# ML 101

## 4. Optimization

Optimization refers to a procedure for finding the input parameters or arguments to a function that result in the minimum or maximum output of the function.

The most common type of optimization problems encountered in machine learning are **continuous function optimization**, where the input arguments to the function are real-valued numeric values, e.g. floating point values. The output from the function is also a real-valued evaluation of the input values.

We might refer to problems of this type as continuous function optimization, to distinguish from functions that take discrete variables and are referred to as combinatorial optimization problems.

There are many different types of optimization algorithms that can be used for continuous function optimization problems, and perhaps just as many ways to group and summarize them.

Perhaps the major division in optimization algorithms is whether the objective function can be differentiated at a point or not. That is, whether the first derivative (gradient or slope) of the function can be calculated for a given candidate solution or not. This partitions algorithms into those that can make use of the calculated gradient information and those that do not:

- Algorithms that use derivative information.
- Algorithms that do not use derivative information.


### Differentiable Objective Function

A differentiable function is a function where the derivative can be calculated for any given point in the input space.

The derivative of a function for a value is the rate or amount of change in the function at that point. It is often called the slope.

- **First-Order Derivative**: Slope or rate of change of an objective function at a given point.

The derivative of the function with more than one input variable (e.g. multivariate inputs) is commonly referred to as the gradient.

- **Gradient**: Derivative of a multivariate continuous objective function.

A derivative for a multivariate objective function is a vector, and each element in the vector is called a partial derivative, or the rate of change for a given variable at the point assuming all other variables are held constant.

- **Partial Derivative**: Element of a derivative of a multivariate objective function.

We can calculate the derivative of the derivative of the objective function, that is the rate of change of the rate of change in the objective function. This is called the second derivative.

- **Second-Order Derivative**: Rate at which the derivative of the objective function changes.

For a function that takes multiple input variables, this is a matrix and is referred to as the Hessian matrix.

- **Hessian matrix**: Second derivative of a function with two or more input variables.

Simple differentiable functions can be optimized analytically using calculus. Typically, the objective functions that we are interested in cannot be solved analytically.

Optimization is significantly easier if the gradient of the objective function can be calculated, and as such, there has been a lot more research into optimization algorithms that use the derivative than those that do not.

#### First-Order Algorithms

First-order optimization algorithms explicitly involve using the first derivative (gradient) to choose the direction to move in the search space.

The procedures involve first calculating the gradient of the function, then following the gradient in the opposite direction (e.g. downhill to the minimum for minimization problems) using a step size (also called the learning rate).

The step size is a hyperparameter that controls how far to move in the search space, unlike “local descent algorithms” that perform a full line search for each directional move.

A step size that is too small results in a search that takes a long time and can get stuck, whereas a step size that is too large will result in zig-zagging or bouncing around the search space, missing the optima completely.

The **Gradient Descent** uses the following update rule:
$$
x_{k+1} = x_{k}-\alpha \times \nabla F(x_k)
$$

In [ ]:
import tqdm
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import celluloid
from celluloid import Camera


class LinearRegressioner(object):
    def __init__(self, w=1, b=1, lr=0.01):
        self.lr = lr
        self.w = np.array([[w]])
        self.b = np.array([b])

    def cost(self, x, y):
        pred = x@self.w + self.b
        e = y-pred
        return np.mean(e**2)

    def fit(self, x, y):
        pred = x@self.w + self.b
        e = y-pred
        dJ_dw = (np.mean(e*(-2*x), axis=0))
        dJ_db = (np.mean(e*(-2), axis=0))
        self.w = (self.w.T - self.lr * dJ_dw).T
        self.b = (self.b - self.lr * dJ_db)

    def predict(self, x):
        return x@self.w.T + self.b

    def params(self):
        return (self.w, self.b)


x_train = np.array([[1], [2], [4], [5], [6], [7]])

y_train = np.array([[4], [-12], [3], [-11], [-5], [-17]])

w_list = []  # for weight values
b_list = []  # for biases
c_list = []  # for cost values
ys_list = []  # prediction on xs
cl_list = []  # y values for x_train

xs = np.array([
    [-3],
    [10]
])

model = LinearRegressioner(w=3, b=-1, lr=0.001)

for i in tqdm.tqdm(range(10000)):
    w_list.append(model.params()[0])
    b_list.append(model.params()[1])
    c_list.append(model.cost(x_train, y_train))
    ys_list.append(model.predict(xs).T)
    cl_list.append(model.predict(x_train).T)
    model.fit(x_train, y_train)


def cost_3d(x, y, w, b):
    pred = x @ w.T + b
    e = y-pred
    return np.mean(e**2)


ws = np.linspace(-5, 5.0, 10)
bs = np.linspace(-5, 5, 10)
M, B = np.meshgrid(ws, bs)

# determine costs for each pair of w and b
# cost_3d() only accepts wp and bp as matrices.
zs = np.array([cost_3d(x_train, y_train, np.array([[wp]]), np.array([[bp]]))
              for wp, bp in zip(np.ravel(M), np.ravel(B))])
Z = zs.reshape(M.shape)

# Define which epochs/data points to plot
a = np.arange(0, 50, 1).tolist()
b = np.arange(50, 100, 5).tolist()
c = np.arange(100, 10000, 200).tolist()
p = a+b+c  # points we want to plot

# Turn lists into arrays
w = np.array(w_list).flatten()
b = np.array(b_list).flatten()
c = np.array(c_list).flatten()
ys = np.array(ys_list)
p = np.array(p)

fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(121)
ax1.set_title("Linear fit", fontsize=30)
ax2 = fig.add_subplot(122, projection='3d')
ax2.set_title("Cost function", fontsize=30)
ax2.view_init(elev=20., azim=145)
camera = Camera(fig)

for i in tqdm.tqdm(p):
    leg = ax1.plot(xs.T.flatten(), ys[i].flatten(), color='r', label=str(i))
    ax1.vlines(x_train.T, ymin=y_train.T,
               ymax=cl_list[i], linestyle="dashed", color='r', alpha=0.3)
    ax1.scatter(x_train, y_train, color='b', marker='x', s=44)
    ax1.legend(leg, [f'epochs: {i}'], loc='upper right', fontsize=15)
    ax1.set_xlabel("x", fontsize=25, labelpad=10)
    ax1.set_ylabel("y", fontsize=25, labelpad=10)
    ax1.tick_params(axis='both', which='major', labelsize=15)
    ax1.set_ylim([-20, 10])

    ax2.plot_surface(M, B, Z, rstride=1, cstride=1, color='b', alpha=0.35)  # create surface plot
    ax2.scatter(w[i], b[i], c[i], marker='o', s=12**2, color='orange')
    ax2.set_xlabel("w", fontsize=25, labelpad=10)
    ax2.set_ylabel("b", fontsize=25, labelpad=10)
    # negative value for labelpad places z-label left of z-axis.
    ax2.set_zlabel("costs", fontsize=25, labelpad=-35)
    ax2.tick_params(axis='both', which='major', labelsize=15)
    ax2.plot(w[0:i], b[0:i], c[0:i], linestyle="dashed", linewidth=2, color="grey")  # (dashed) lineplot

    plt.tight_layout()
    camera.snap()

animation = camera.animate(interval=5, repeat=False, repeat_delay=500)
animation.save('../figs/SimpleLinReg_3.gif', writer='imagemagick')


![Gradient descent](../figs/SimpleLinReg_3.gif)

#### Second-Order Algorithms

Second-order optimization algorithms explicitly involve using the second derivative (Hessian) to choose the direction to move in the search space.

These algorithms are only appropriate for those objective functions where the Hessian matrix can be calculated or approximated.

The **Netwon Method** uses the following update rule:
$$
x_{k+1} = x_k - \frac{f'(x_k)}{f''(x_k)}
$$

![Newton Method](../figs/NewtonIteration_Ani.gif)

### Non-Differential Objective Function

Optimization algorithms that make use of the derivative of the objective function are fast and efficient.

Nevertheless, there are objective functions where the derivative cannot be calculated, typically because the function is complex for a variety of real-world reasons. Or the derivative can be calculated in some regions of the domain, but not all, or is not a good guide.

Some difficulties on objective functions for the classical algorithms described in the previous section include:
- No analytical description of the function (e.g. simulation).
- Multiple global optima (e.g. multimodal).
- Stochastic function evaluation (e.g. noisy).
- Discontinuous objective function (e.g. regions with invalid solutions).

As such, there are optimization algorithms that do not expect first- or second-order derivatives to be available.

These algorithms are sometimes referred to as black-box optimization algorithms as they assume little or nothing (relative to the classical methods) about the objective function.

#### Stochastic Algorithms

Stochastic optimization algorithms are algorithms that make use of randomness in the search procedure for objective functions for which derivatives cannot be calculated.

Unlike the deterministic direct search methods, stochastic algorithms typically involve a lot more sampling of the objective function, but are able to handle problems with deceptive local optima.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import optimization.sa as sa

plt.rcParams['figure.figsize'] = [10, 5]

# define objective function
def f(x):
    return x**2

def cost(x):
    return np.power(x, 2)[0]

# plot the function
x = np.linspace(-5,5,100)
y = f(x)

plt.plot(x, y)
plt.show()

In [ ]:
bounds = np.asarray([(-5.0, 5.0)])
solution = sa.simulated_annealing(cost, bounds, n_iter=1000, debug=True)
print(f'F({solution[0]})={f(solution[0])}')

plt.plot(solution[2])
plt.show()

#### Population Algorithms

Population optimization algorithms are stochastic optimization algorithms that maintain a pool (a population) of candidate solutions that together are used to sample, explore, and hone in on an optima.

Algorithms of this type are intended for more challenging objective problems that may have noisy function evaluations and many global optima (multimodal), and finding a good or good enough solution is challenging or infeasible using other methods.

The pool of candidate solutions adds robustness to the search, increasing the likelihood of overcoming local optima.

In [ ]:
import optimization.de as de
bounds = np.asarray([(-5.0, 5.0)])
solution = de.differential_evolution(cost, bounds, n_iter=10, debug=True)
print(f'F({solution[0]})={f(solution[0])}')

obj_best_iter, obj_avg_iter, obj_worst_iter = solution[2]

plt.plot(obj_best_iter)
plt.plot(obj_avg_iter)
plt.plot(obj_worst_iter)
plt.show()